In [1]:
%pip install beautifulsoup4 haystack-ai

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from bs4 import BeautifulSoup

## List Data Files

In [3]:
base_directory = "Data"

In [4]:
def _list_htm_files():
    """
    Recursively finds all .htm files in the base directory and its subdirectories.
        
    Returns:
        list: A list of file paths relative to the base directory.
    """
    htm_files = []
    for root, _, files in os.walk(base_directory):
        for file in files:
            if file.endswith(".htm"):
                relative_path = os.path.relpath(os.path.join(root, file), start=base_directory)
                htm_files.append(os.path.join(base_directory, relative_path))
    
    return htm_files

In [5]:
htm_files = _list_htm_files() # DEMO: Listing all the htm_files in the Data folder. 

In [6]:
htm_files

['Data\\Contact_us.htm',
 'Data\\csh-redirect.htm',
 'Data\\First_Topic.htm',
 'Data\\Help_Missing.htm',
 'Data\\index.htm',
 'Data\\topic.htm',
 'Data\\Computed_Curve_Templates\\Drilling\\Cement_Volume.htm',
 'Data\\Computed_Curve_Templates\\Drilling\\D_Exponent.htm',
 'Data\\Computed_Curve_Templates\\Drilling\\Mechanical_Specific_Energy.htm',
 'Data\\Computed_Curve_Templates\\Drilling\\Temperature_Gradient.htm',
 'Data\\Computed_Curve_Templates\\Gas\\Formation_Fluid_Evaluation\\C1_Sum.htm',
 'Data\\Computed_Curve_Templates\\Gas\\Formation_Fluid_Evaluation\\Gas_Balance.htm',
 'Data\\Computed_Curve_Templates\\Gas\\Formation_Fluid_Evaluation\\Gas_Character.htm',
 'Data\\Computed_Curve_Templates\\Gas\\Formation_Fluid_Evaluation\\Gas_Wetness.htm',
 'Data\\Computed_Curve_Templates\\Gas\\Formation_Fluid_Evaluation\\Inverse_Oil_Indicator.htm',
 'Data\\Computed_Curve_Templates\\Gas\\Formation_Fluid_Evaluation\\Oil_Indicator.htm',
 'Data\\Computed_Curve_Templates\\Gas\\Formation_Fluid_Evaluati

## Process Data Files

In [7]:
%pip install pandas tabulate

Note: you may need to restart the kernel to use updated packages.


In [8]:
import re
import pandas as pd
from tabulate import tabulate

In [9]:
def extract_text(soup):
    # Define navigation-related keyword patterns
    navigation_keywords = [
        r'contact\s+us', r'click\s+(here|for)', r'guidance', r'help', r'support', r'assistance',
        r'maximize\s+screen', r'view\s+details', r'read\s+more', r'convert.*file', r'FAQ', r'learn\s+more'
    ]
    
    navigation_pattern = re.compile(r"|".join(navigation_keywords), re.IGNORECASE)

    # Remove navigation-related text
    for tag in soup.find_all("p"):
        if navigation_pattern.search(tag.text):
            tag.decompose()

    # Extract only meaningful paragraph text (excluding very short ones)
    paragraphs = [p.get_text(strip=True) for p in soup.find_all("p") if len(p.get_text(strip=True)) > 20]
    
    clean_text = "\n\n".join(paragraphs)
    
    return clean_text

In [10]:
def extract_list(soup):
    # Extract lists properly
    lists = []
    for ul in soup.find_all("ul"):
        items = [li.get_text(strip=True) for li in ul.find_all("li")]
        lists.append(items)
    return lists

In [11]:
def extract_table_as_text_block(file_path):
    """
    Extract tables from HTML as a single formatted text block for inclusion into page_text.
    Skips navigation tables and handles no-table cases.

    Args:
        file_path (str): Path to the file (for metadata).

    Returns:
        str: Formatted block of all tables from this file, or a message if no tables are found.
    """
    try:
        tables = pd.read_html(file_path)

        def is_navigation_table(table):
            """Detect if table is a 'navigation-only' table with just 'back' and 'forward'."""
            flattened = [str(cell).strip().lower() for cell in table.to_numpy().flatten()]
            navigation_keywords = {"back", "forward"}
            return set(flattened).issubset(navigation_keywords)
        
        def is_nan_only_table(table):
            """Detect if the entire table only contains NaN values."""
            return table.isna().all().all()

        table_texts = []
        table_count = 0

        for idx, table in enumerate(tables):
            if is_navigation_table(table) or is_nan_only_table(table):
                continue
            
            if table.shape[1] == 2:
                # Drop rows where both the second and third columns are NaN
                table = table.dropna(how='all')

                last_col = table.columns[-1]

                table[last_col] = table[last_col].fillna("")

            table_count += 1
            formatted_table = tabulate(table, headers="keys", tablefmt="grid")

            beautified_table = f"""
╔════════════════════════════════════════════════════╗
║            📊 Table {table_count} from {file_path}              ║
╚════════════════════════════════════════════════════╝

{formatted_table}

╔════════════════════════════════════════════════════╗
║            🔚 End of Table {table_count}                       ║
╚════════════════════════════════════════════════════╝
"""
            table_texts.append(beautified_table)

        if not table_texts:
            return ""

        return "\n".join(table_texts)

    except ValueError:
        # No tables found case
        return ""

In [12]:
%pip install lxml html5lib

Note: you may need to restart the kernel to use updated packages.


In [13]:
import logging 
from haystack import Document

In [14]:
def load_content():
    """
    Load and process all .htm files from the given base directory for information retrieval.
    
    Returns:
        list: A list of dictionaries containing processed text content and metadata.
    """

    # Find all .htm files in the directory
    htm_files = _list_htm_files()
    logging.info(f"Found {len(htm_files)} .htm files.")

    documents = []
    page_texts = []

    for file_path in htm_files:
        try:
            with open(file_path, encoding="utf-8") as file:
                content = file.read()

                # Extract content inside the <body> tag
                content = content[content.find("<body>") + 6 : content.find("</body>")]

                soup = BeautifulSoup(content, "html.parser")

                # Extract text, tables, and lists
                clean_text = extract_text(soup)
                formatted_table = extract_table_as_text_block(file_path)
                lists = extract_list(soup)

                # Combine extracted content
                page_text = "\n".join(filter(None, [clean_text, formatted_table, "\n".join(["• " + item for sublist in lists for item in sublist])]))

                page_texts.append(page_text)

                # Store the processed content as a Document Objects. 
                document = Document(content=page_text, meta={"source": os.path.basename(file_path)})

                documents.append(document)

        except UnicodeDecodeError:
            logging.error(f"Could not read the file {file_path}. Check the file encoding.")

    logging.info(f"Processed {len(documents)} documents.")
    
    return documents

In [15]:
from bs4 import XMLParsedAsHTMLWarning
import warnings

warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

In [16]:
training_documents = load_content()

C:\Users\HaochengLin\AppData\Local\Temp\ipykernel_26024\174411885.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table[last_col] = table[last_col].fillna("")


## Document Query and Retrieval

In [ ]:
%pip install sentence-transformers ollama-haystack

In [ ]:
from haystack import Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack_integrations.components.embedders.ollama import OllamaTextEmbedder
from haystack_integrations.components.embedders.ollama import OllamaDocumentEmbedder
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever

In [ ]:
document_store = InMemoryDocumentStore(embedding_similarity_function="cosine")

document_embedder = OllamaDocumentEmbedder()
documents_with_embeddings = document_embedder.run(training_documents)['documents']
document_store.write_documents(documents_with_embeddings, policy="overwrite")

In [ ]:
query_pipeline = Pipeline()
query_pipeline.add_component("text_embedder", OllamaTextEmbedder())
query_pipeline.add_component("retriever", InMemoryEmbeddingRetriever(document_store=document_store))
query_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")

query = "What is the limit to the number of curves in GEO?"

result = query_pipeline.run({"text_embedder":{"text": query}})

In [ ]:
result

## Similarity Ranking Document Query

In [ ]:
%pip install transformers[torch,sentencepiece]

In [ ]:
from haystack import Document
from haystack.components.rankers import TransformersSimilarityRanker

In [ ]:
docs = [Document(content="Paris"), Document(content="Berlin")]

In [ ]:
ranker = TransformersSimilarityRanker()
ranker.warm_up()

In [ ]:
ranker.run(query="City in France", documents=docs, top_k=1)

In [ ]:
ranker.run(query="I have 20000 modifiers added ty log, why I can't I add anymore?", documents=training_documents, top_k=1)

## Ollama Model (Offline)

In [17]:
%pip install langchain-ollama

Note: you may need to restart the kernel to use updated packages.


In [18]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate

In [19]:
offline_model = OllamaLLM(model="llama3.2:latest")

In [20]:
template = """
You are an AI assistant designed to help users navigate the GEO application.

**Context:**  
GEO is a well log authoring, analysis, and reporting system for petroleum geologists, geoscientists, and engineers.  
Answer the user's question using **only** the provided documents.  

**Instructions:**  
- Use information from the **Documents** section to generate your response.  
- Provide a **direct**, **concise**, and **factual** answer.  
- **Avoid** speculative or unnecessary **explanations** or **justifications**.  
- If the question is about a **numerical** or a **limit-based** constraint, return only the limit and its enforcement.  

---
**Documents:**  
{documents}  
---
**Question:** {question}  
---

**Your Optimized Answer:**  
"""

In [21]:
prompt = ChatPromptTemplate.from_template(template)

In [22]:
# Read the file
with open("processed_content.txt", "r", encoding="utf-8") as file:
    content = file.read()

In [23]:
chain = prompt | offline_model

In [24]:
chain.invoke({
    "documents": content,
    "question": "I have 20000 modifiers added ty log, why I can't I add anymore?"
})

'If you are unable to add more than 20,000 modifiers to the log in GEO, it is likely due to a limitation on the maximum number of characters allowed for a single modifier. This limitation typically depends on the size of the data and the available storage space.\n\n**Possible Causes:**\n\n1. **Maximum Character Limit**: Each modifier may have a specific character limit, which could be exceeded when trying to add more modifiers.\n2. **Data Storage Space**: The log file might not have enough free space to accommodate additional modifiers, causing the system to reject new additions.\n3. **System Configuration**: Some system settings or configuration options might restrict the number of available slots for adding new modifiers.\n\n**Troubleshooting Steps:**\n\n1. Check the character limit for each modifier and ensure it does not exceed the allowed value.\n2. Verify that there is sufficient free space in the log file to accommodate additional modifiers.\n3. Adjust any system settings or con

In [25]:
response = chain.invoke({
    "documents": content,
    "question": "What's the maximum curves I can load in a data file?"
})

In [26]:
print(response)

The text does not provide information on the maximum number of curves that can be loaded in a data file.


In [27]:
chain.invoke({
    "documents": content,
    "question": "What are the limits to the number of tracks?"
})

"There is no information provided about the limits to the number of tracks in the text. The only relevant information is related to saving and sharing ODF files, as well as the use of GEOView and GEOe-View. \n\nHowever, I can provide some general information that might be relevant:\n\n* In GEO, track layouts are saved in VEW files when they contain presentation data such as layout and formatting.\n* There is no explicit limit mentioned on the number of tracks per file, but it's likely that there may be performance or technical limitations.\n\nIf you're looking for a more definitive answer, I would recommend checking the documentation or support resources provided by Geologix Limited, as they may have specific information on this topic."